In [ ]:
!pip install pymorphy2
!pip install stanfordnlp
!pip install nltk
!pip install numpy
!pip install sklearn
!pip install pandas
!pip install spacy==2.3
!pip install razdel

In [ ]:
!wget https://github.com/buriy/spacy-ru/releases/download/v2.3_beta/ru2_grameval_300.zip
!unzip ru2_grameval_300.zip

In [ ]:
!pip show spacy

In [4]:
import spacy
nlp = spacy.load('ru2_grameval_300')

In [ ]:
import nltk
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
import pymorphy2 as py

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

stopwords = stopwords.words('russian')


In [9]:
abz = u'''
Президентом РФ предложены конституционные поправки, направленные на усиление социальных гарантий гражданам РФ 
и уточнение полномочий органов властиВ частности, гражданам РФ гарантируется оплата труда на уровне не ниже 
прожиточного минимума в целом по РФ, а также регулярная индексация пенсий. К ведению Российской Федерации 
предлагается отнести установление ограничений для замещения государственных и муниципальных должностей, 
должностей государственной и муниципальной службы, в том числе и ограничений, связанных с наличием иностранного 
гражданства или вида на жительство. Госдуме предоставляется право утверждать кандидатов на должности Заместителей 
Председателя Правительства РФ и федеральных министров.Указывается, что кандидаты на должности руководителей 
федеральных "силовых" ведомств, а также прокуроров назначаются после консультаций с Советом Федерации.Президент 
РФ наделяется полномочиями по внесению в Совет Федерации представлений о прекращении полномочий судей 
Конституционного Суда РФ, Верховного Суда РФ, судей кассационных и апелляционных судов, в том числе в случае 
совершения ими порочащего честь и достоинство судьи проступка.Закрепляется статус Государственного Совета РФ 
как органа, обеспечивающего согласованное функционирование и взаимодействие органов государственной власти, 
определение основных направлений внутренней и внешней политики РФ и приоритетных направлений социально-экономического 
развития государства.Устанавливается, что органы местного самоуправления и органы государственной власти входят 
в единую систему публичной власти в РФ.Вводится механизм проверки конституционности положений федерального закона, 
принятого Госдумой и одобренного Советом Федерации. Такая проверка проводится Конституционным Судом РФ по запросу 
Президента РФ. В случае установления неконституционности закона он возвращается Президентом РФ в Госдуму без 
подписания.Устанавливается требование о необходимости вынесения настоящего Закона о поправке к Конституции РФ на 
всенародное голосование.'''

import re
from razdel import sentenize

abz = re.sub(r'\s+', ' ', abz)
abz = re.sub(r'([а-я])([А-Я](\s|[А-Я]+|[а-я]+))', r'\1 \2', abz)
abz = re.sub(r'([а-я])\.([А-Я](\s|[А-Я]+|[а-я]+))', r'\1. \2', abz)
abz = re.sub(r'([А-Я]{2,})\.([А-Я](\s|[А-Я]+|[а-я]+))', r'\1. \2', abz)

sent_list = set()

for sent in sentenize(abz):
    doc = nlp(sent.text)

    for tok in doc:
        sent_list.add(' '.join([str(s) for s in reversed([tok] + list(tok.ancestors))]))
print(sent_list)


{'предложены поправки направленные усиление на', 'Закрепляется статус органа как', 'предлагается отнести установление ограничений замещения ограничений связанных наличием гражданства', 'наделяется полномочиями внесению представлений прекращении случае', 'предлагается отнести установление ограничений замещения для', 'Устанавливается требование необходимости вынесения поправке голосование', 'возвращается Президентом РФ', 'возвращается случае установления неконституционности закона', 'наделяется полномочиями внесению представлений прекращении', 'предложены гарантируется оплата индексация ,', 'наделяется полномочиями внесению представлений прекращении случае в', 'наделяется полномочиями внесению представлений прекращении случае совершения честь достоинство судьи проступка', 'Закрепляется статус органа', 'предложены гарантируется оплата уровне ниже минимума целом', 'предложены поправки направленные усиление гарантий социальных', 'предоставляется Госдуме', 'Устанавливается входят систему РФ 

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd

# collecting tf-idf for all data in no_stop_words

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import os

morph = py.MorphAnalyzer()
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

index_value = {}
transformed = []
row_transformed = []

raw_docs = ''
for sent in sent_list:
  raw_docs+='. '+sent

docs = nltk.tokenize.sent_tokenize(raw_docs)
print(docs)

transformed = tfidf_vec.fit_transform(docs)
index_value = {i[1]: i[0] for i in tfidf_vec.vocabulary_.items()}

fully_indexed = {}
for row in transformed:
    fully_indexed.update({
        index_value[column]: value for (column, value) in zip(row.indices, row.data) if int(value * 100) >= 40})
sorted_dict = list(fully_indexed.items())
sorted_dict.sort(key=lambda i: i[1])
print(sorted_dict[::-1][1:10])

['.', 'предложены поправки направленные усиление на.', 'Закрепляется статус органа как.', 'предлагается отнести установление ограничений замещения ограничений связанных наличием гражданства.', 'наделяется полномочиями внесению представлений прекращении случае.', 'предлагается отнести установление ограничений замещения для.', 'Устанавливается требование необходимости вынесения поправке голосование.', 'возвращается Президентом РФ.', 'возвращается случае установления неконституционности закона.', 'наделяется полномочиями внесению представлений прекращении.', 'предложены гарантируется оплата индексация ,.', 'наделяется полномочиями внесению представлений прекращении случае в. наделяется полномочиями внесению представлений прекращении случае совершения честь достоинство судьи проступка.', 'Закрепляется статус органа.', 'предложены гарантируется оплата уровне ниже минимума целом.', 'предложены поправки направленные усиление гарантий социальных.', 'предоставляется Госдуме.', 'Устанавливается 

In [20]:
all_data = []

for term in sorted_dict:
    tokenized = word_tokenize(term[0].lower())
    all_data.append(''.join(term[0]) + '\n')

In [ ]:
termin_list = []
for term in all_data:
    terms = word_tokenize(term.lower())
    print(terms)
    if len(terms) == 1:
        print(len(terms))
        for word in terms:
            parsed = morph.parse(word.lower())[0]
            if parsed.tag.POS == "NOUN" and [word] not in termin_list:
                print(word)
                termin_list.append([word])
            elif parsed.tag.POS == "VERB" or parsed.tag.POS == "INFN" and [word] not in termin_list:
                print(word)
                termin_list.append([word])
            else:
                print('not a term')
                break
    elif len(terms) == 2:
        print(len(terms))
        for word in terms:
            parsed = morph.parse(word.lower())[0]
            file_base_name = 0
            if parsed.tag.POS == "NOUN" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1])
                termin_list.append([word, terms[file_base_name + 1]])
                break
            elif parsed.tag.POS == "ADJS" or parsed.tag.POS == "ADJF" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1])
                termin_list.append([word, terms[file_base_name + 1]])
                break
            else:
                print('not a term')
                break
    else:
        print(len(terms))
        for word in terms:
            parsed = morph.parse(word.lower())[0]
            file_base_name = 0
            if parsed.tag.POS == "NOUN" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "CONJ" or morph.parse(terms[file_base_name + 1])[
                0].tag.POS == "PREP" or \
                    morph.parse(terms[file_base_name + 1])[0].tag.POS == "PRCL" and morph.parse(terms[file_base_name + 2])[0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1], terms[file_base_name + 2])
                termin_list.append([word, terms[file_base_name + 1], terms[file_base_name + 2]])
                break
            elif parsed.tag.POS == "VERB" or parsed.tag.POS == "INFN" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "NOUN" and \
                    morph.parse(terms[file_base_name + 2])[
                        0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1], terms[file_base_name + 2])
                termin_list.append([word, terms[file_base_name + 1], terms[file_base_name + 2]])
                break
            elif parsed.tag.POS == "NOUN" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "ADJS" or morph.parse(terms[file_base_name + 1])[
                0].tag.POS == "ADJF" and \
                    morph.parse(terms[file_base_name + 2])[0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1], terms[file_base_name + 2])
                termin_list.append([word, terms[file_base_name + 1], terms[file_base_name + 2]])
                break
            elif parsed.tag.POS == "NOUN" and morph.parse(terms[file_base_name + 1])[0].tag.POS == "NOUN" and morph.parse(terms[file_base_name + 2])[
                0].tag.POS == "NOUN":
                print(word, terms[file_base_name + 1], terms[file_base_name + 2])
                termin_list.append([word, terms[file_base_name + 1], terms[file_base_name + 2]])
                break
            else:
                print('not a term')
                break

print(termin_list)